# TP2 Robotica Movel - Potential Fields

## Alunos
- Bernardo Nogueira Borges 2020006396
- Daniele Cassia Silva Diniz 2020076874


### Importando Libs

In [2]:
from coppeliasim_zmqremoteapi_client import RemoteAPIClient
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import networkx as nx
import numpy as np

client = RemoteAPIClient()
sim = client.require('sim')
np.set_printoptions(precision=3,suppress=True)



### Definindo Parametros

In [8]:
def disableObject(object_name):
    object = sim.getObject(object_name)
    if object == -1:
        print(f"No object with name {object_name} was found.")
        return
    
    def disableComponents(handle):
        # Invisible
        sim.setObjectInt32Param(handle,sim.objintparam_visibility_layer,0)

        # No Dynamics
        # sim.setObjectInt32Param(handle,sim.shapeintparam_static,1)

        i = 0
        while True:
            child_handle = sim.getObjectChild(handle, i)
            if child_handle == -1:
                break
            disableComponents(child_handle)
            i += 1
    
    # Disable the object and recursive
    disableComponents(object)
    print(f"Object {object_name} and all components were disabled.")

# Desativar Robotino (Maybe)



Object /robotino and all components were disabled.


In [8]:

CREATE_PLOTS = False

### Obtendo Obstaculos da Imagem

In [4]:
pioneer = sim.getObject("/Pioneer_p3dx")

def getRobotQ():
    robotM = sim.getObjectMatrix(pioneer)
    return [robotM[3],robotM[7],math.atan2(robotM[1],robotM[0])]

q = getRobotQ()
print(f"q = {q}")

def matrix_rel_robot(name):
    obj = sim.getObject(f"/{name}")
    M = sim.getObjectMatrix(obj,pioneer)
    M = np.array(M).reshape(-1,4)
    return M



q = [-0.7444717884063721, -2.3749983310699463, -1.4835309872482196]


### Criando Gradiente

## Funções Potencial

### Força de Atração
$$F_{att}(q) = k_{att} \cdot (q_{goal} - q)$$

In [30]:
def ForceAttraction(q, q_goal, katt=.01):
    return katt * (q_goal - q)

### Força de repulsão
$$F_{rep,i}(q) = -\nabla U_{rep,i}(q)$$
$$=\begin{cases}
  \begin{aligned}
    \frac{k_{rep,i}}{\rho_i^2(q)}\left( \frac{1}{\rho_i(q)} - \frac{1}{\rho_{0,i}} \right)^{\gamma-1} \frac{q - q_{obs,i}}{\rho_i(q)} &, \textnormal{se } \rho_i(q) \le \rho_{0,i}\\
    0&, \textnormal{se } \rho_i(q) \gt \rho_{0,i}\\
  \end{aligned}
\end{cases}
$$

In [31]:
LIMIT_RANGE = 5

def ForceRepulsion(q, q_obs_i, dist_i, krep=.1):
    if dist_i > LIMIT_RANGE:
        return 0
    
    ans = (krep / dist_i**2)
    ans *= (1/dist_i) - (1/LIMIT_RANGE)
    ans *= (q-q_obs_i) / dist_i 

    return ans


### Força Resultante 
$$F(q)  = F_{att}(q) + \sum_{i=1}^{p}{F_{rep,i}(q)}$$

In [34]:
def ForceResult(q,q_goal,q_obs,dist):
    ans = ForceAttraction(q,q_goal) 
    for i in range(len(q_obs)):
        ans += ForceRepulsion(q,q_obs[i],dist[i])
    return ans


### Controladores
- [De Luca e Oriolo, 1994](https://www.researchgate.net/publication/225543929_Control_of_Wheeled_Mobile_Robots_An_Experimental_Overview)
$$v = k_{\rho}(\dot x \cos \theta + \dot y \sin \theta)$$
$$\omega = k_{\theta}(atan2(\dot y,\dot x) - \theta)$$

### Definindo a Leitura do Lazer

In [38]:
def readSensorData():
    # ? Atualizacao da funcao simxGetStringSignal 
    string_range_data = sim.getStringSignal("hokuyo_range_data")
    string_angle_data = sim.getStringSignal("hokuyo_angle_data")

    # Verifica se os dados foram obtidos corretamente
    if string_range_data == None or string_angle_data == None: return None
    
    # unpack dos dados de range e angulos do sensor
    raw_range_data = sim.unpackFloatTable(string_range_data)
    raw_angle_data = sim.unpackFloatTable(string_angle_data)
    return raw_range_data, raw_angle_data

def getObstaclePositions(laser_data):
    # Matriz de transformação laser -> robô
    Trl = matrix_rel_robot("fastHokuyo")
    Trl = np.vstack([Trl,[0,0,0,1]])

    # Matriz de transformação robô -> mundo
    Twr = sim.getObjectMatrix(pioneer)
    Twr = np.vstack([np.array(Twr).reshape(3,4),[0,0,0,1]])

    def laser_read_to_point(laser_read):
        ang, dist = laser_read
        x = dist * np.cos(ang)
        y = dist * np.sin(ang)
                
        # Ponto de leitura no referencial do laser
        Pl = np.array([x,y,0,1])
        # ponto laser -> robô
        Pr =  Trl @ Pl
        # ponto robô -> mundo
        Pw =  Twr @ Pr
        return np.array([Pw[0],Pw[1]])

    return list(map(laser_read_to_point,laser_data))


def draw_laser_data(laser_data,ax, max_sensor_range=5,show_robot=True):

    # Matriz de transformação laser -> robô
    Trl = matrix_rel_robot("fastHokuyo")
    Trl = np.vstack([Trl,[0,0,0,1]])

    # Matriz de transformação robô -> mundo
    Twr = sim.getObjectMatrix(pioneer)
    Twr = np.vstack([np.array(Twr).reshape(3,4),[0,0,0,1]])

 
    for i in range(len(laser_data)):
        ang, dist = laser_data[i]
        
        # Quando o feixe não acerta nada, retorna o valor máximo (definido na simulação)
        # Logo, usar um pequeno limiar do máximo para considerar a leitura
        if (max_sensor_range - dist) < 0.1:
            continue

        x = dist * np.cos(ang)
        y = dist * np.sin(ang)

        if show_robot:
            c = 'b' if ang < 0 else 'r'
        else:
            c = 'g'
        
        # Ponto de leitura no referencial do laser
        Pl = np.array([x,y,0,1])
        # ponto laser -> robô
        Pr =  Trl @ Pl
        # ponto robô -> mundo
        Pw =  Twr @ Pr

        ax.plot(Pw[0], Pw[1], 'o', color=c)

    if show_robot:
        # Valores globais do Robo
        q = getRobotQ()

        # Plot do robô com referencial global
        ax.plot(q[0], q[1], marker=(3, 0, q[2]*90), markersize=20, linestyle='None', color='black')

    ax.grid()
    ax.set_xlim([-max_sensor_range, max_sensor_range])
    ax.set_ylim([-max_sensor_range, max_sensor_range])

### Movendo o Robo

In [39]:
sim.startSimulation()

# Prosseguindo com as leituras
# Geralmente a primeira leitura é inválida (atenção ao Operation Mode)
# Em loop até garantir que as leituras serão válidas
sensor_data = readSensorData()
while sensor_data == None:
    sensor_data = readSensorData()

raw_range_data, raw_angle_data = sensor_data
laser_data = np.array([raw_angle_data, raw_range_data]).T

q = getRobotQ()
print(q)
q_goal = np.array([1,2])
q_obs = getObstaclePositions(laser_data)

if CREATE_PLOTS:
    fig, ax = plt.subplots(figsize=(6, 6), dpi=100)
    ax.set_aspect('equal')
    ax.grid()
    draw_laser_data(laser_data,ax)

force = ForceResult(q[:2],q_goal,q_obs,dist=raw_range_data)
print(force)

while (t := sim.getSimulationTime()) < 1:
    sim.step()

sim.stopSimulation()


[-0.743331522715727, -2.3870381110277514, -1.489867278883126]
[15.867 -6.73 ]
